In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 참고 사항
# 공공 부문의 임금은 15일과 마지막 날 2주에 한 번씩 지급
# 슈퍼마켓은 영향을 받을 수 밖에 없음

# 2016년 4월 16일 에콰도르에서 규모 7.8의 지진 발생
# 사람들은 지진 발생 후 몇 주 동안 슈퍼마켓 판매에 큰 영향을 미친 물과 다른 필요한 제품을 기부하는 구호 활동에 모임

# 데이터 모델 

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
train = pd.read_csv('../input/new-dataset/new_train.csv',  low_memory=False)
train.drop(['Unnamed: 0'], axis=1, inplace=True)
print(train.shape)
train.head()

In [11]:
test = pd.read_csv('../input/new-dataset/new_test.csv')
test.drop(['Unnamed: 0'], axis=1, inplace=True)
print(test.shape)
test.head()

In [12]:
submission = pd.read_csv('../input/new-dataset/new_submission.csv', index_col='id')
submission.drop(['Unnamed: 0'], axis=1, inplace=True)
print(submission.shape)
submission.head()

In [13]:
train.info()

# Encoding(인코딩)

In [14]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i+1  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    print(label_maps)
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

le = make_label_map(train)
train = label_encoder(train, le)
train.head()

In [15]:
test.info()

In [16]:
le = make_label_map(test)
test = label_encoder(test, le)
test.head()

In [17]:
train = train.loc[100001:180000]

In [18]:
print(train.shape)
train.head()

In [19]:
# 넘파이 변환
X = train.drop(['sales'], axis = 1).values
y = train['sales'].values 
X.shape, y.shape

In [20]:
# 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    
                                                    random_state=1
                                                    )
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [21]:
# 스케일링 - 정규화 진행
from sklearn.preprocessing import StandardScaler

# z-score
sc_x = StandardScaler() # X에 대한 정규화 mean, std가 정해지지 않음
sc_y = StandardScaler() # y에 대한 정규화 mean, std가 정해지지 않음
sc_x.fit(X_train) # 인풋 X_train에 대한 mean, std 정해주기 
sc_y.fit(y_train.reshape(-1,1))

In [22]:
# 훈련 데이터 정규화
X_train = sc_x.fit_transform(X_train)
y_train = sc_y.fit_transform( np.expand_dims(y_train, 1) ).flatten()
print(X_train.shape, y_train.shape)

In [23]:
# 테스트 데이터 정규화
X_test = sc_x.fit_transform(X_test)
y_test = sc_y.fit_transform( np.expand_dims(y_test, 1) ).flatten()
print(X_test.shape, y_test.shape)

# AutoML

In [1]:
# Main pycaret
!pip install --ignore-installed pycaret

In [2]:
!pip install imbalanced-learn==0.7.0

In [3]:
!pip uninstall sklearn -y
!pip install --upgrade sklearn
!pip install scikit-learn==0.23.2 --user

In [4]:
import sklearn
sklearn.__version__
import sys
sys.version

In [5]:
!pip install ipykernel==4.10
!pip install ipython==5.5.0
!pip install tornado==5.1.0,
!pip install pydeck

In [6]:
!pip install pycaret[full]
!pip install Jinja2==3.1.2 # xgboost
!pip install catboost

In [8]:
from pycaret.regression import *

In [24]:
data = setup(data=train, target='sales',train_size=0.8)

In [25]:
compare_models()

In [26]:
lgb = create_model('rf')
lgb = create_model('et')
lgb = create_model('xgboost')
lgb = create_model('catboost')
lgb = create_model('lightgbm')
lgb = create_model('dt')

In [27]:
# 앙상블
blender = blend_models(estimator_list=compare_models(n_select=3))

In [28]:
final = finalize_model(blender)

In [31]:
test.rename(columns = {'dat_name':'day_name'}, inplace=True)

In [32]:
y_pred = predict_model(final, data=test)
y_pred

In [33]:
y_pred.shape,submission.shape

In [ ]:
y_pred.columns

In [35]:
pred = y_pred['Label']
submission['sales']=pred
submission.head()

In [36]:
submission.head()

In [37]:
submission.to_csv('AutoML[30,000].csv', index=False)